# Textual entailment

### Imports

In [2]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sys
display_step = 10

### Embeddings

Download Glove word embeddings

In [51]:
glove_zip_file = "data/glove.6B.zip"
glove_vectors_file = "data/glove.6B.50d.txt"
import zipfile, urllib.request, shutil, os
    
#large file - 862 MB
if (not os.path.isfile(glove_zip_file) and
    not os.path.isfile(glove_vectors_file)):
    with urllib.request.urlopen("http://nlp.stanford.edu/data/glove.6B.zip") as response, open(glove_zip_file, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

Unzip word embeddings

In [4]:
import zipfile
def unzip_single_file(zip_file_name, output_file_name):
    """
        If the outFile is already created, don't recreate
        If the outFile does not exist, create it from the zipFile
    """
    if not os.path.isfile(output_file_name):
        with open(output_file_name, 'wb') as out_file:
            with zipfile.ZipFile(zip_file_name) as zipped:
                for info in zipped.infolist():
                    if output_file_name in info.filename:
                        with zipped.open(info) as requested_file:
                            out_file.write(requested_file.read())
                            return

unzip_single_file(glove_zip_file, glove_vectors_file)

In [52]:
glove_wordmap = {}
with open(glove_vectors_file, "r", encoding="utf8") as glove:
    for line in glove:
        name, vector = tuple(line.split(" ", 1))
        glove_wordmap[name] = np.fromstring(vector, sep=" ")

KeyboardInterrupt: 

## Model

### Embed sentences - sentence2sequence

In [4]:
def sentence2sequence(sentence):
    """
    - Turns an input sentence into an (n,d) matrix, 
        where n is the number of tokens in the sentence
        and d is the number of dimensions each word vector has.
    
      Tensorflow doesn't need to be used here, as simply
      turning the sentence into a sequence based off our 
      mapping does not need the computational power that
      Tensorflow provides. Normal Python suffices for this task.
    """
    tokens = sentence.lower().split(" ")
    rows = []
    words = []
    #Greedy search for tokens
    for token in tokens:
        i = len(token)
        while len(token) > 0 and i > 0:
            word = token[:i]
            if word in glove_wordmap:
                rows.append(glove_wordmap[word])
                words.append(word)
                token = token[i:]
                i = len(token)
            else:
                i = i-1
    return rows, words

In [5]:
display_step = 1
# one hot encoding
def score_setup(row):
    convert_dict = {
      'ENTAILMENT': 0,
      'NEUTRAL': 1,
      'CONTRADICTION': 2
    }
    score = np.zeros((3,))
    tag = row["entailment_judgment"]
    score[convert_dict[tag]] += 1
    return score

def fit_to_size(matrix, shape):
    res = np.zeros(shape)
    slices = [slice(0,min(dim,shape[e])) for e, dim in enumerate(matrix.shape)]
    res[slices] = matrix[slices]
    return res

### Cell used for development

In [56]:
def create_model():
    tf.reset_default_graph() 
    data_feature_list, correct_values, correct_scores = split_data_into_scores()

    l_h, l_e = max_hypothesis_length, max_evidence_length
    N, D, H = batch_size, vector_size, hidden_size
    l_seq = l_h + l_e
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    lstm_drop =  tf.contrib.rnn.DropoutWrapper(lstm, input_p, output_p)
    hyp = tf.placeholder(tf.float32, [N, l_h, D], 'hypothesis')
    evi = tf.placeholder(tf.float32, [N, l_e, D], 'evidence')
    y = tf.placeholder(tf.float32, [N, 3], 'label')
    lstm_back = tf.contrib.rnn.BasicLSTMCell(lstm_size)

    lstm_drop_back = tf.contrib.rnn.DropoutWrapper(lstm_back, input_p, output_p)

    fc_initializer = tf.random_normal_initializer(stddev=0.1) 
    fc_weight = tf.get_variable('fc_weight', [2*hidden_size, 3], 
                            initializer = fc_initializer)
    fc_bias = tf.get_variable('bias', [3])
    tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, 
                     tf.nn.l2_loss(fc_weight)) 

    x = tf.concat([hyp, evi], 1) # N, (Lh+Le), d
    x = tf.transpose(x, [1, 0, 2]) # (Le+Lh), N, d
    x = tf.reshape(x, [-1, vector_size]) # (Le+Lh)*N, d
    x = tf.split(x, l_seq,)
    rnn_outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm, lstm_back,
                                                            x, dtype=tf.float32)

    classification_scores = tf.matmul(rnn_outputs[-1], fc_weight) + fc_bias

    with tf.variable_scope('Accuracy'):
        predicts = tf.cast(tf.argmax(classification_scores, 1), 'int32')
        y_label = tf.cast(tf.argmax(y, 1), 'int32')
        corrects = tf.equal(predicts, y_label)
        num_corrects = tf.reduce_sum(tf.cast(corrects, tf.float32))
        accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

    with tf.variable_scope("loss"):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            logits = classification_scores, labels = y)
        loss = tf.reduce_mean(cross_entropy)
        total_loss = loss + weight_decay * tf.add_n(
            tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    opt_op = optimizer.minimize(total_loss)
    # Initialize variables
    init = tf.global_variables_initializer()

    # Use TQDM if installed
    tqdm_installed = False
    try:
        from tqdm import tqdm
        tqdm_installed = True
    except:
        pass

    # Launch the Tensorflow session
    sess = tf.Session()
    sess.run(init)

    # training_iterations_count: The number of data pieces to train on in total
    # batch_size: The number of data pieces per batch
    training_iterations = range(0,training_iterations_count,batch_size)
    if tqdm_installed:
        # Add a progress bar if TQDM is installed
        training_iterations = tqdm(training_iterations)

    for i in training_iterations:
        if i % 1000 == 0:
        # Select indices for a random data subset
            batch = np.random.randint(data_feature_list[0].shape[0], size=batch_size)

        # Use the selected subset indices to initialize the graph's 
        #   placeholder values
        hyps, evis, ys = (data_feature_list[0][batch,:],
                          data_feature_list[1][batch,:],
                          correct_scores[batch])

        # Run the optimization with these initialized values
        sess.run([opt_op], feed_dict={hyp: hyps, evi: evis, y: ys})
        # display_step: how often the accuracy and loss should 
        #   be tested and displayed.
        if (i/batch_size) % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={hyp: hyps, evi: evis, y: ys})
            # Calculate batch loss
            tmp_loss = sess.run(loss, feed_dict={hyp: hyps, evi: evis, y: ys})
            # Display results
            print("Iter " + str(i/batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(tmp_loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    data_feature_list, correct_values, correct_scores = split_test_data_into_scores("data/dev.txt")


    hyps, evis, ys = (data_feature_list[0][:],
                      data_feature_list[1][:],
                      correct_scores)
    predictions = sess.run(classification_scores, feed_dict={hyp: hyps, evi: evis, y: ys})
    total = len(predictions)
    correct_predictions = 0
    for i,prediction in enumerate(predictions):
        if np.argmax(prediction[0])==np.argmax(ys[i]):
            correct_predictions += 1
    #     print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
    #       " / " + ["Positive", "Neutral", "Negative"][np.argmax(ys[i])])
    acc = correct_predictions*100/total
    print("Acc: %s" % str(acc))
    return acc

In [57]:
import numpy as np
accs = [] # 128
batches = [128, 64, 32, 1, 256]
for batch in batches:
    batch_size = batch
    accs.append(create_model())

print(accs)
print("Best batch size %s" % str(batches[np.argmax(accs)]))


f:\python 36 64\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.




  0%|                                                                                                                                                                                   | 0/782 [00:00<?, ?it/s]

Iter 0.0, Minibatch Loss= 0.956232, Training Accuracy= 0.58594






  0%|▏                                                                                                                                                                          | 1/782 [00:00<10:43,  1.21it/s]



  1%|▊                                                                                                                                                                          | 4/782 [00:00<07:39,  1.69it/s]



  1%|█▌                                                                                                                                                                         | 7/782 [00:01<05:30,  2.35it/s]



  1%|██▏                                                                                                                                                                       | 10/782 [00:01<04:00,  3.21it/s]

Iter 10.0, Minibatch Loss= 0.938195, Training Accuracy= 0.58594






  2%|██▌                                                                                                                                                                       | 12/782 [00:01<03:01,  4.23it/s]



  2%|███▎                                                                                                                                                                      | 15/782 [00:01<02:16,  5.64it/s]



  2%|███▉                                                                                                                                                                      | 18/782 [00:01<01:44,  7.31it/s]

...





  9%|███████████████▋                                                                                                                                                          | 72/782 [00:04<00:31, 22.68it/s]



 10%|████████████████▎                                                                                                                                                         | 75/782 [00:04<00:29, 23.72it/s]



 10%|████████████████▉                                                                                                                                                         | 78/782 [00:04<00:29, 23.97it/s]

Iter 80.0, Minibatch Loss= 0.917919, Training Accuracy= 0.58594






 10%|█████████████████▌                                                                                                                                                        | 81/782 [00:04<00:30, 22.76it/s]



 11%|██████████████████▎                                                                                                                                                       | 84/782 [00:04<00:29, 23.87it/s]



 11%|██████████████████▉                                                                                                                                                       | 87/782 [00:04<00:28, 24.68it/s]



 12%|███████████████████▌                                                                                                                                                      | 90/782 [00:04<00:27, 24.96it/s]

Iter 90.0, Minibatch Loss= 0.935087, Training Accuracy= 0.58594


KeyboardInterrupt: 

### Constants

In [ ]:
#Constants setup
max_hypothesis_length, max_evidence_length = 28, 32
batch_size, vector_size, hidden_size = 128, 50, 256

training_iterations_count = 100000

lstm_size = hidden_size

weight_decay = 0.001

learning_rate = 1

input_p, output_p = 0.6, 0.3

### [Training data](http://www.site.uottawa.ca/~diana/csi5386/A2_2019/SICK_train.txt)

In [53]:
import numpy as np

def split_data_into_scores(file_name="data/training.txt"):
    import csv
    with open(file_name,"r") as data:
        train = csv.DictReader(data , delimiter='\t')
        evi_sentences = []
        hyp_sentences = []
        labels = []
        scores = []
        count = 1
        for row in train:
            hyp_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_A"].lower())[0]))
            evi_sentences.append(np.vstack(
                    sentence2sequence(row["sentence_B"].lower())[0]))
            labels.append(row["entailment_judgment"])
            scores.append(score_setup(row))
        
        hyp_sentences = np.stack([fit_to_size(x, (max_hypothesis_length, vector_size))
                          for x in hyp_sentences])
        evi_sentences = np.stack([fit_to_size(x, (max_evidence_length, vector_size))
                          for x in evi_sentences])
                                 
        return (hyp_sentences, evi_sentences), labels, np.array(scores)
data_feature_list, correct_values, correct_scores = split_data_into_scores()

l_h, l_e = max_hypothesis_length, max_evidence_length
N, D, H = batch_size, vector_size, hidden_size
l_seq = l_h + l_e

f:\python 36 64\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


### TRAINING

In [54]:
tf.reset_default_graph()

In [55]:
lstm = tf.contrib.rnn.LSTMCell(lstm_size)
lstm_drop =  tf.contrib.rnn.DropoutWrapper(lstm, input_p, output_p)

In [56]:
# N: The number of elements in each of our batches, 
#   which we use to train subsets of data for efficiency's sake.
# l_h: The maximum length of a hypothesis, or the second sentence.  This is
#   used because training an RNN is extraordinarily difficult without 
#   rolling it out to a fixed length.
# l_e: The maximum length of evidence, the first sentence.  This is used
#   because training an RNN is extraordinarily difficult without 
#   rolling it out to a fixed length.
# D: The size of our used GloVe or other vectors.
hyp = tf.placeholder(tf.float32, [N, l_h, D], 'hypothesis')
evi = tf.placeholder(tf.float32, [N, l_e, D], 'evidence')
y = tf.placeholder(tf.float32, [N, 3], 'label')
# hyp: Where the hypotheses will be stored during training.
# evi: Where the evidences will be stored during training.
# y: Where correct scores will be stored during training.

# lstm_size: the size of the gates in the LSTM, 
#    as in the first LSTM layer's initialization.
# lstm_back = tf.contrib.rnn.BasicLSTMCell(lstm_size)
lstm_back = tf.contrib.rnn.LSTMCell(lstm_size)

# lstm_back:  The LSTM used for looking backwards 
#   through the sentences, similar to lstm.

# input_p: the probability that inputs to the LSTM will be retained at each
#   iteration of dropout.
# output_p: the probability that outputs from the LSTM will be retained at 
#   each iteration of dropout.
lstm_drop_back = tf.contrib.rnn.DropoutWrapper(lstm_back, input_p, output_p)
# lstm_drop_back:  A dropout wrapper for lstm_back, like lstm_drop.


fc_initializer = tf.random_normal_initializer(stddev=0.1) 
# fc_initializer: initial values for the fully connected layer's weights.
# hidden_size: the size of the outputs from each lstm layer.  
#   Multiplied by 2 to account for the two LSTMs.
fc_weight = tf.get_variable('fc_weight', [2*hidden_size, 3], 
                            initializer = fc_initializer)
# fc_weight: Storage for the fully connected layer's weights.
fc_bias = tf.get_variable('bias', [3])
# fc_bias: Storage for the fully connected layer's bias.

# tf.GraphKeys.REGULARIZATION_LOSSES:  A key to a collection in the graph
#   designated for losses due to regularization.
#   In this case, this portion of loss is regularization on the weights
#   for the fully connected layer.
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, 
                     tf.nn.l2_loss(fc_weight)) 

x = tf.concat([hyp, evi], 1) # N, (Lh+Le), d
# Permuting batch_size and n_steps
x = tf.transpose(x, [1, 0, 2]) # (Le+Lh), N, d
# Reshaping to (n_steps*batch_size, n_input)
x = tf.reshape(x, [-1, vector_size]) # (Le+Lh)*N, d
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
x = tf.split(x, l_seq,)

# x: the inputs to the bidirectional_rnn


# tf.contrib.rnn.static_bidirectional_rnn: Runs the input through
#   two recurrent networks, one that runs the inputs forward and one
#   that runs the inputs in reversed order, combining the outputs.
rnn_outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm, lstm_back,
                                                            x, dtype=tf.float32)
# rnn_outputs: the list of LSTM outputs, as a list. 
#   What we want is the latest output, rnn_outputs[-1]

classification_scores = tf.matmul(rnn_outputs[-1], fc_weight) + fc_bias
# The scores are relative certainties for how likely the output matches
#   a certain entailment: 
#     0: Positive entailment
#     1: Neutral entailment
#     2: Negative entailment

In [57]:
with tf.variable_scope('Accuracy'):
    predicts = tf.cast(tf.argmax(classification_scores, 1), 'int32')
    y_label = tf.cast(tf.argmax(y, 1), 'int32')
    corrects = tf.equal(predicts, y_label)
    num_corrects = tf.reduce_sum(tf.cast(corrects, tf.float32))
    accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

with tf.variable_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        logits = classification_scores, labels = y)
    loss = tf.reduce_mean(cross_entropy)
    total_loss = loss + weight_decay * tf.add_n(
        tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

opt_op = optimizer.minimize(total_loss)

In [ ]:
# Initialize variables
init = tf.global_variables_initializer()

from tqdm import tqdm
import time
start = time.time()
# Launch the Tensorflow session
sess = tf.Session()
sess.run(init)

# training_iterations_count: The number of data pieces to train on in total
# batch_size: The number of data pieces per batch
training_iterations = range(0,training_iterations_count,batch_size)
print(training_iterations)
training_iterations = tqdm(training_iterations)
for j in range(10):
    for i in training_iterations:
        if i % 1000 == 0:
        # Select indices for a random data subset
            batch = np.random.randint(data_feature_list[0].shape[0], size=batch_size)

        # Use the selected subset indices to initialize the graph's 
        #   placeholder values
        hyps, evis, ys = (data_feature_list[0][batch,:],
                          data_feature_list[1][batch,:],
                          correct_scores[batch])

        # Run the optimization with these initialized values
        sess.run([opt_op], feed_dict={hyp: hyps, evi: evis, y: ys})
        # display_step: how often the accuracy and loss should 
        #   be tested and displayed.
        if (i/batch_size) % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={hyp: hyps, evi: evis, y: ys})
            # Calculate batch loss
            tmp_loss = sess.run(loss, feed_dict={hyp: hyps, evi: evis, y: ys})
            # Display results
            print("Iter " + str(i/batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(tmp_loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
stop = time.time()
mins = int((stop-start)/60.0)
print("Training took %s:%s" % (str(mins), str(stop - start - mins*60)))

range(0, 100000, 128)


  0%|                                                                          | 0/782 [00:00<?, ?it/s]

Iter 0.0, Minibatch Loss= 0.982853, Training Accuracy= 0.53125


  0%|                                                                  | 1/782 [00:03<41:27,  3.18s/it]

Iter 1.0, Minibatch Loss= 1.072385, Training Accuracy= 0.35156


...

Iter 102.0, Minibatch Loss= 0.863691, Training Accuracy= 0.54688
Iter 103.0, Minibatch Loss= 0.689754, Training Accuracy= 0.66406
Iter 104.0, Minibatch Loss= 0.551138, Training Accuracy= 0.73438
Iter 105.0, Minibatch Loss= 0.525222, Training Accuracy= 0.80469
Iter 106.0, Minibatch Loss= 0.533901, Training Accuracy= 0.76562
Iter 107.0, Minibatch Loss= 0.641196, Training Accuracy= 0.77344
Iter 108.0, Minibatch Loss= 0.622623, Training Accuracy= 0.75000
Iter 109.0, Minibatch Loss= 0.778973, Training Accuracy= 0.64844
Iter 110.0, Minibatch Loss= 0.848506, Training Accuracy= 0.55469
Iter 111.0, Minibatch Loss= 0.754497, Training Accuracy= 0.65625
Iter 112.0, Minibatch Loss= 0.789867, Training Accuracy= 0.70312
Iter 113.0, Minibatch Loss= 0.785745, Training Accuracy= 0.67969
Iter 114.0, Minibatch Loss= 0.828806, Training Accuracy= 0.68750
Iter 115.0, Minibatch Loss= 0.836599, Training Accuracy= 0.62500
Iter 116.0, Minibatch Loss= 0.808418, Training Accuracy= 0.73438
Iter 117.0, Minibatch Los

Iter 228.0, Minibatch Loss= 0.067442, Training Accuracy= 0.99219
Iter 229.0, Minibatch Loss= 0.060960, Training Accuracy= 0.99219
Iter 230.0, Minibatch Loss= 0.056558, Training Accuracy= 0.99219
Iter 231.0, Minibatch Loss= 0.052922, Training Accuracy= 0.99219
Iter 232.0, Minibatch Loss= 0.049785, Training Accuracy= 0.99219
Iter 233.0, Minibatch Loss= 0.046909, Training Accuracy= 0.99219
Iter 234.0, Minibatch Loss= 0.044161, Training Accuracy= 0.99219
Iter 235.0, Minibatch Loss= 0.041413, Training Accuracy= 0.99219
Iter 236.0, Minibatch Loss= 0.038497, Training Accuracy= 0.99219
Iter 237.0, Minibatch Loss= 0.035181, Training Accuracy= 0.99219
Iter 238.0, Minibatch Loss= 0.031335, Training Accuracy= 0.99219
Iter 239.0, Minibatch Loss= 0.027438, Training Accuracy= 1.00000
Iter 240.0, Minibatch Loss= 0.024088, Training Accuracy= 1.00000
Iter 241.0, Minibatch Loss= 0.021374, Training Accuracy= 1.00000
Iter 242.0, Minibatch Loss= 0.019246, Training Accuracy= 1.00000
Iter 243.0, Minibatch Los

Iter 354.0, Minibatch Loss= 0.930155, Training Accuracy= 0.57812
Iter 355.0, Minibatch Loss= 0.935706, Training Accuracy= 0.58594
Iter 356.0, Minibatch Loss= 0.923116, Training Accuracy= 0.57031
Iter 357.0, Minibatch Loss= 0.942102, Training Accuracy= 0.58594
Iter 358.0, Minibatch Loss= 0.941677, Training Accuracy= 0.52344
Iter 359.0, Minibatch Loss= 0.969321, Training Accuracy= 0.58594
Iter 360.0, Minibatch Loss= 0.943069, Training Accuracy= 0.54688
Iter 361.0, Minibatch Loss= 0.931666, Training Accuracy= 0.58594
Iter 362.0, Minibatch Loss= 0.907328, Training Accuracy= 0.57031
Iter 363.0, Minibatch Loss= 0.911445, Training Accuracy= 0.58594
Iter 364.0, Minibatch Loss= 0.900997, Training Accuracy= 0.57812
Iter 365.0, Minibatch Loss= 0.918441, Training Accuracy= 0.58594
Iter 366.0, Minibatch Loss= 0.917092, Training Accuracy= 0.53906
Iter 367.0, Minibatch Loss= 0.961588, Training Accuracy= 0.58594
Iter 368.0, Minibatch Loss= 0.949750, Training Accuracy= 0.56250
Iter 369.0, Minibatch Los

Iter 480.0, Minibatch Loss= 0.803023, Training Accuracy= 0.58594
Iter 481.0, Minibatch Loss= 0.826462, Training Accuracy= 0.54688
Iter 482.0, Minibatch Loss= 0.919301, Training Accuracy= 0.57031
Iter 483.0, Minibatch Loss= 1.304534, Training Accuracy= 0.45312
Iter 484.0, Minibatch Loss= 0.903109, Training Accuracy= 0.52344
Iter 485.0, Minibatch Loss= 0.859716, Training Accuracy= 0.54688
Iter 486.0, Minibatch Loss= 0.818197, Training Accuracy= 0.54688
Iter 487.0, Minibatch Loss= 0.784455, Training Accuracy= 0.55469
Iter 488.0, Minibatch Loss= 0.776278, Training Accuracy= 0.55469
Iter 489.0, Minibatch Loss= 0.837015, Training Accuracy= 0.54688
Iter 490.0, Minibatch Loss= 1.052018, Training Accuracy= 0.53125
Iter 491.0, Minibatch Loss= 1.022600, Training Accuracy= 0.50781
Iter 492.0, Minibatch Loss= 0.964819, Training Accuracy= 0.57031
Iter 493.0, Minibatch Loss= 0.840342, Training Accuracy= 0.48438
Iter 494.0, Minibatch Loss= 0.796339, Training Accuracy= 0.57031
Iter 495.0, Minibatch Los

Iter 606.0, Minibatch Loss= 0.997203, Training Accuracy= 0.45312
Iter 607.0, Minibatch Loss= 1.030998, Training Accuracy= 0.54688
Iter 608.0, Minibatch Loss= 0.980409, Training Accuracy= 0.43750
Iter 609.0, Minibatch Loss= 0.993240, Training Accuracy= 0.54688
Iter 610.0, Minibatch Loss= 0.930692, Training Accuracy= 0.43750
Iter 611.0, Minibatch Loss= 0.945407, Training Accuracy= 0.55469
Iter 612.0, Minibatch Loss= 0.888636, Training Accuracy= 0.53906
Iter 613.0, Minibatch Loss= 0.941746, Training Accuracy= 0.55469
Iter 614.0, Minibatch Loss= 0.818176, Training Accuracy= 0.55469
Iter 615.0, Minibatch Loss= 0.861705, Training Accuracy= 0.56250
Iter 616.0, Minibatch Loss= 0.725093, Training Accuracy= 0.62500
Iter 617.0, Minibatch Loss= 0.719015, Training Accuracy= 0.56250
Iter 618.0, Minibatch Loss= 0.725397, Training Accuracy= 0.58594
Iter 619.0, Minibatch Loss= 0.719792, Training Accuracy= 0.57031
Iter 620.0, Minibatch Loss= 0.774898, Training Accuracy= 0.59375
Iter 621.0, Minibatch Los

### Developing

In [188]:
# import numpy as np
# accs = [] # 128
# batches = [128, 64, 32, 1, 256]
# for batch in batches:
#     batch_size = batch
#     print("dev batch %s" % str(batch))
#     accs.append(test("data/dev.txt"))

# print(accs)
# print("Best batch size %s" % str(batches[np.argmax(accs)]))


### Testing

In [ ]:
def test(file_test="data/test_labeled.txt"):
    data_feature_list, correct_values, correct_scores = split_data_into_scores(file_test)

    #hyps, evis, ys 
    data = (data_feature_list[0],
                      data_feature_list[1],
                      correct_scores)
    # predictions = []
    correct_predictions = 0
    total_predictions = 0
    for i, f1 in enumerate(data_feature_list[0]):
        hyps, evis, ys = data_feature_list[0][i], data_feature_list[1][i], correct_scores[i]
        prediction = sess.run(classification_scores, feed_dict={hyp: ([hyps] * N), evi: ([evis] * N), y: ([ys] * N)})
    #     predictions.append(prediction)
        total_predictions += 1
        if np.argmax(prediction[0])==np.argmax(ys):
            correct_predictions += 1
    #     if total_predictions % 500 == 0:
    #         print(total_predictions)
    #     print(["Positive", "Neutral", "Negative"][np.argmax(prediction[0])]+
    #       " / " + ["Positive", "Neutral", "Negative"][np.argmax(ys[i])])
    print(total_predictions)

    # acc = sess.run(accuracy)#, feed_dict={'label': correct_scores})
    acc = correct_predictions*100/total_predictions
    print("Acc: %s" % str(acc))    
# print("Acc: %s" % str(acc))

import time
start = time.time()
test()
stop = time.time()
mins = int((stop-start)/60.0)
print("Training took %s:%s" % (str(mins), str(stop - start - mins*60)))

In [190]:
sess.close()
#56.52